In [1]:
import sys
sys.path.append('../')

In [2]:
from viz import visualize_token_probabilities

from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
torch_model = GPT2LMHeadModel.from_pretrained("gpt2")

input_text = "Once we've built an abstraction layer that makes the training job independent of the GPU, we can absolutely compete in this market. We can do even better by analyzing the workflows and the types of jobs people are actually submitting. This will allow us to determine which chip is best optimized to run those workflows. We can then build that chip and seamlessly switch it out for our customers."

input_ids = tokenizer(input_text, return_tensors="pt").input_ids

print(input_ids[0].tolist())

# visualize individual tokens
input_text_tokens = [tokenizer.decode([token_id]) for token_id in input_ids[0]]
print(input_text_tokens)
print(len(input_text_tokens))

[7454, 356, 1053, 3170, 281, 34651, 7679, 326, 1838, 262, 3047, 1693, 4795, 286, 262, 11362, 11, 356, 460, 5543, 9320, 287, 428, 1910, 13, 775, 460, 466, 772, 1365, 416, 22712, 262, 670, 44041, 290, 262, 3858, 286, 3946, 661, 389, 1682, 24353, 13, 770, 481, 1249, 514, 284, 5004, 543, 11594, 318, 1266, 23392, 284, 1057, 883, 670, 44041, 13, 775, 460, 788, 1382, 326, 11594, 290, 33681, 5078, 340, 503, 329, 674, 4297, 13]
['Once', ' we', "'ve", ' built', ' an', ' abstraction', ' layer', ' that', ' makes', ' the', ' training', ' job', ' independent', ' of', ' the', ' GPU', ',', ' we', ' can', ' absolutely', ' compete', ' in', ' this', ' market', '.', ' We', ' can', ' do', ' even', ' better', ' by', ' analyzing', ' the', ' work', 'flows', ' and', ' the', ' types', ' of', ' jobs', ' people', ' are', ' actually', ' submitting', '.', ' This', ' will', ' allow', ' us', ' to', ' determine', ' which', ' chip', ' is', ' best', ' optimized', ' to', ' run', ' those', ' work', 'flows', '.', ' We', ' 

In [3]:
from models.gpt2 import GPT2
from states import to_jax_state
import jax
import jax.numpy as jnp
input_ids = jnp.array(input_ids)
print(input_ids)
model = GPT2()
state = to_jax_state(torch_model)
rng = jax.random.PRNGKey(0)

all_probs = model.calc_token_probs(state, input_ids, rng, only_last_token=False)
print(all_probs.shape)


[[ 7454   356  1053  3170   281 34651  7679   326  1838   262  3047  1693
   4795   286   262 11362    11   356   460  5543  9320   287   428  1910
     13   775   460   466   772  1365   416 22712   262   670 44041   290
    262  3858   286  3946   661   389  1682 24353    13   770   481  1249
    514   284  5004   543 11594   318  1266 23392   284  1057   883   670
  44041    13   775   460   788  1382   326 11594   290 33681  5078   340
    503   329   674  4297    13]]
(77, 50257)


In [4]:
# get the probabilities for each token

input_text_probs = jnp.array([all_probs[i, token_id] for i, token_id in enumerate(input_ids[0])])
print(input_text_probs)
print(input_text_probs.shape)


[1.2537574e-05 8.4897634e-05 9.3415765e-06 3.1882835e-05 4.3287643e-04
 5.7088269e-04 2.7531164e-03 3.1674127e-04 9.4960469e-06 3.5311759e-04
 3.8261327e-04 2.6138342e-04 3.0396670e-05 1.8042295e-04 3.8444292e-04
 5.3091971e-03 4.8380338e-05 6.6033979e-05 6.8431567e-05 9.1220856e-05
 5.5659376e-07 2.5482549e-04 6.8873844e-05 1.9714785e-04 5.6684012e-06
 4.9931895e-08 5.3555657e-05 1.4588675e-04 9.4391980e-05 4.2844451e-05
 1.3722587e-04 2.0262711e-05 2.5930669e-04 1.0089484e-04 8.1502258e-06
 7.7949936e-04 4.7728588e-04 2.0617363e-06 5.8773119e-05 1.8843237e-04
 7.5746328e-05 3.0457506e-05 7.4682385e-04 7.3171395e-06 4.6565319e-06
 3.4417249e-07 2.9431059e-05 1.1658506e-06 1.1803000e-05 2.5741223e-04
 2.9424780e-06 5.1131428e-05 2.3721746e-04 1.2925660e-04 1.5015308e-04
 9.1690317e-06 3.1463377e-04 2.9645680e-05 3.7928188e-05 9.3784067e-08
 2.8527660e-07 4.7923818e-06 3.3022964e-08 8.3242361e-05 6.5147984e-03
 2.4733754e-04 4.1006343e-04 1.5660404e-05 3.2562847e-04 5.9194129e-04
 4.296

In [5]:
visualize_token_probabilities(input_text, input_text_probs, tokenizer)